Transport Equation

Half radius bubble centered at $x_c$:
$\phi(x) = \frac{1}{{1 + e^{(||x-x_c||−0.5)/\epsilon }}}$

Level set advection equation:
$\phi_t +\nabla\cdot(\phi u) = 0$

Reinitialisation:
$\phi_\tau (x, \tau) + \nabla \cdot \left[\phi(x, \tau)(1 - \phi(x, \tau))\hat{n}\right] = \varepsilon \nabla \cdot \left[\hat{n}(\nabla \phi(x, \tau) \cdot \hat{n})\right]$

where
$\hat{n} = \hat{n}(x) = \frac{\nabla \phi(x, \tau = 0)}{|\nabla \phi(x, \tau = 0)|}.$

Calculating this in a constant $u$ field, thus no bubble deformation.

Weak form for advection equation:
$(\phi_t, v)=(\phi, \nabla v \cdot u)$


In [4]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista
import matplotlib.pyplot as plt

from dolfinx import default_scalar_type
from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical, locate_dofs_topological
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter, XDMFFile
from dolfinx.mesh import create_unit_square, exterior_facet_indices, locate_entities_boundary, locate_entities, meshtags, Mesh
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction, as_matrix,as_vector, SpatialCoordinate, Measure,
                 div, dot, ds, dx, inner, lhs, nabla_grad,grad, rhs, sym)

In [5]:
grid_length = 10
t = 0
T = 4
num_steps = 1000
dt = T / num_steps
mesh = create_unit_square(MPI.COMM_WORLD, grid_length, grid_length)

v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
V = functionspace(mesh, v_cg2)
W = functionspace(mesh, s_cg1)
Vr, submap = V.sub(0).collapse()

boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], 1)),
              (3, lambda x: np.isclose(x[1], 0)),
              (4, lambda x: np.isclose(x[1], 1))]

facet_indices, facet_markers = [], []
fdim = mesh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(mesh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(mesh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)

# Velocity Conditions
sym_ur_dofs = locate_dofs_topological(V.sub(0),fdim,facet_tag.find(1))
sym_ur = Function(Vr)
bc_sym_ur = dirichletbc(PETSc.ScalarType(0), sym_ur_dofs, V.sub(0))

bcu = [bc_sym_ur]

In [ ]:
n = FacetNormal(mesh)
k = Constant(mesh, PETSc.ScalarType(dt))
u = Function(V)
h = 1/grid_length
d = 0.1


def unit_vector(x):
    values = np.ones((2, x.shape[1]), dtype=PETSc.ScalarType)
    values[0] = 0
    return values

u.interpolate(unit_vector)
w = TestFunction(W)
x = SpatialCoordinate(mesh)

phi_star = TrialFunction(W)
eps =  h**(1 - d) / 2
phi_n = Function(W)

def bubble(x):
    return 1/(1+np.exp( (np.sqrt((x[0]**2+(x[1]-0.5)**2))-0.25)/eps))

phi_n.interpolate(bubble)

#Time loop
for step in range(num_steps):
    t += dt
    print(t)

    #Euler in Time
    F = -dot(phi_star, w)*x[0]*dx
    F += dot(phi_n, w)*x[0]*dx
    F += k*dot(phi_n*u, grad(w))*x[0]*dx

    a1 = form(lhs(F))
    L1 = form(rhs(F))
    problem_1 = LinearProblem(a1, L1, bcu, phi_star, petsc_options={"ksp_type": "minres", "pc_type": "hypre"}) #Maybe solving the wrong thing ?
    problem_1.solve()

    # Need to compute normals of phi_star
    n_star = TrialFunction(V)
    n = Function(V)
    N = dot(n_star,w)*x[0]*dx - dot(grad(phi_star), w)*x[0]*dx
    an = form(lhs(N))
    Ln = form(rhs(N))
    problem_n = LinearProblem(an, Ln, [], n_star, petsc_options={"ksp_type": "minres", "pc_type": "hypre"})
    problem_n.solve()
    n.interpolate(n_star)
    
    #Reinitialisation
    dtau =  h**(1 + d) / 2
    max_iteration =100
    tol = 1e-10
    
    for i in range(max_iteration):

        R = inner(phi_n1 - phi_star, w) *x[0]*dx
        R -= dtau * (0.5*(phi_star + phi_n1) - phi_star*phi_n1) * inner(n, grad(w)) *x[0]*dx
        R += dtau * eps * inner(grad(0.5*(phi_star + phi_n1)), n) * inner(n, grad(w)) *x[0]*dx
        a2 = form(lhs(R))
        L2 = form(rhs(R))
        problem_2 = LinearProblem(a2, L2, [], phi_n1, petsc_options={"ksp_type": "minres", "pc_type": "hypre"})

        prev = np.copy(phi_star.x.array)
        problem_2.solve()
        if np.linalg.norm(phi_n1.x.array - phi_star.x.array) < tol:
            phi_n.x.array[:] = phi_n1.x.array[:]
            print(i + 1)
            break
        else:
            phi_star.x.array[:] = phi_n1.x.array[:]
            phi_n.x.array[:] = phi_n1.x.array[:]
        




0.004


NameError: name 'LinearProblem' is not defined